# Get EKS (baseline) deployment times

In [8]:
eks_deployment=!awk '/^Time/ {print $2}' ../logs/eks/*16gb_deployment*
print(eks_deployment)

['1164', '1175', '1100', '1018', '1154', '1089', '1038', '1144', '980', '1101']


# Get EKS (baseline) shutdown times

In [9]:
eks_shutdown=!awk '/^Time/ {print $2}' ../logs/eks/*16gb_shutdown*
print(eks_shutdown)

['590', '529', '529', '567', '530', '672', '541', '551', '551', '565']


# Get kOps 8x c5.2xlarge deployment times

In [10]:
kops_8x_8vcpu_deployment=!awk '/^Time/ {print $2}' ../logs/kops/*16gb_deployment*
print(kops_8x_8vcpu_deployment)

['306', '272', '252', '297', '243', '294', '304', '295', '795', '256']


# Get kOps 8x c5.2xlarge shutdown times

In [11]:
kops_8x_8vcpu_shutdown=!awk '/^Time/ {print $2}' ../logs/kops/*16gb_shutdown*
print(kops_8x_8vcpu_shutdown)

['107', '97', '97', '88', '96', '98', '97', '107', '107', '108']


# Get kOps 8x c5.18xlarge deployment times

In [12]:
kops_1x_72vcpu_deployment=!awk '/^Time/ {print $2}' ../logs/kops/*144gb_deployment*
print(kops_1x_72vcpu_deployment)

['238', '258', '256', '254', '235', '288', '278', '262', '304', '255']


# Get kOps 8x c5.18xlarge shutdown times

In [13]:
kops_1x_72vcpu_shutdown=!awk '/^Time/ {print $2}' ../logs/kops/*144gb_shutdown*
print(kops_1x_72vcpu_shutdown)

['113', '107', '89', '99', '108', '77', '108', '96', '91', '107']
